<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_04/7%20-%20C2_W4_Lab_1_multi_class_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Classificador multiclasse

Neste laboratório, você verá como criar um modelo para distinguir entre mais de duas classes.

O código será semelhante ao que você usou anteriormente, com algumas alterações importantes no modelo e nos parâmetros de treinamento.

## Faça o download e prepare o conjunto de dados

Você usará o [_Rock-Paper-Scissors dataset_](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors), uma galeria de imagens de mãos em poses de pedra, papel e tesoura.

In [ ]:
# Faça o download do conjunto de treino
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps.zip

# Faça o download do conjunto de teste
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps-test-set.zip

In [ ]:
import zipfile

# Extrair o arquivo
local_zip = './rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/rps-train')
zip_ref.close()

local_zip = './rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/rps-test')
zip_ref.close()

Como de costume, você atribuirá os nomes dos diretórios a variáveis e examinará os nomes dos arquivos como uma verificação.

In [ ]:
import os

base_dir = 'tmp/rps-train/rps'

rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

print('Total de imagens de pedra:', len(os.listdir(rock_dir)))
print('Total de imagens de papel:', len(os.listdir(paper_dir)))
print('Total de imagens de tesoura:', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

Você também pode inspecionar algumas das imagens para ver a variedade das entradas do modelo.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

## Construir o modelo

Em seguida, você criará sua CNN.

Você usará 4 camadas de convolução com filtros 64-64-128-128 e, em seguida, anexará uma camada `Dropout` para evitar o _overfitting_ e algumas camadas densas para a classificação.

A camada de saída seria uma camada densa de 3 neurônios ativada por [Softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax).

Você já viu antes quando estava treinando com o Fashion MNIST. Ela dimensiona sua saída para um conjunto de probabilidades que somam 1.

A ordem dessa saída de 3 neurônios seria `papel`-`pedra`-`tesoura` (por exemplo, uma saída `[0,8 0,2 0,0]` significa que o modelo está prevendo 80% de probabilidade para papel e 20% de probabilidade para pedra.

Você pode examinar a arquitetura com `model.summary()` abaixo.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Observe que a forma de entrada é o tamanho desejado da imagem 150x150 com 3 bytes de cor
    # Essa é a primeira convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # A segunda convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A terceira convolução
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quarta convolução
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Achatar os resultados para alimentar um DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # Camada oculta de 512 neurônios
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Imprimir o resumo do modelo
model.summary()

Em seguida, você compilará o modelo.

A principal mudança aqui é a função `loss`.

Antes você estava usando `binary_crossentropy` para 2 classes, mas agora alterará para [categorical_crossentropy](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-function) para estendê-la a mais classes.

In [ ]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## Preparar o ImageDataGenerator

Você preparará os geradores como antes.

Você definirá o conjunto de treinamento para o aumento de dados para que ele possa imitar outras poses que o modelo precisa aprender.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = "tmp/rps-train/rps"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "tmp/rps-test/rps-test-set"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

## Treine o modelo e avalie os resultados

Você treinará por 25 épocas e avaliará os resultados em seguida.

Observe como a acurácia do treinamento e da validação estão tendendo para cima.

Essa é uma boa indicação de que o modelo não está se sobreajustando ao conjunto de treinamento.

In [ ]:
# Treinar o modelo
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Acurácia de Treino')
plt.plot(epochs, val_acc, 'b', label='Acurácia de Validação')
plt.title('Acurácia de Treino e Validação')
plt.legend(loc=0)
plt.figure()

plt.show()

# Previsão de modelo

Você deve conseguir fazer upload de uma imagem aqui e classificá-la sem travamentos.

**OBSERVAÇÃO: Esse bloco de código só funcionará no Google Colab.**

Você pode usar suas próprias imagens ou usar as que estão disponíveis [aqui](https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps-validation.zip)

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  path = fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

## Encerramento

Isso conclui este breve exercício sobre os classificadores multiclasse.

Você viu que, com apenas algumas alterações, foi possível converter seus classificadores binários para prever mais classes.

Você usou as mesmas técnicas de preparação de dados e modelos e conseguiu obter resultados relativamente bons em apenas 25 épocas. 

Para praticar, você pode procurar outros conjuntos de dados (por exemplo, [aqui](https://archive.ics.uci.edu/datasets)) com mais classes e revisar o modelo para acomodá-los.

Tente fazer experimentos com diferentes camadas e técnicas de aumento de dados para melhorar suas métricas.